**Get all the coordinates first**


In [29]:
#Get the coordintes of the searchbar, first starting row and close button
import pyautogui
import time

def capture_coordinate_after_delay(delay=5):
    """Waits for a specified delay and then captures the current mouse coordinates."""
    print(f"Move your mouse to the desired coordinate. Capturing in {delay} seconds...")
    time.sleep(delay)
    x, y = pyautogui.position()
    print(f"Coordinate captured: ({x}, {y})")

capture_coordinate_after_delay()

Move your mouse to the desired coordinate. Capturing in 5 seconds...
Coordinate captured: (233, 244)


In [1]:
SEARCH_BAR = (1131, 130)
CLOSE_BUTTON = (1210, 130)  
STARTING_ROW = (710, 208)  
ROW_HEIGHT = 20

TOP_LEFT_X, TOP_LEFT_Y = (189, 208)
BOTTOM_RIGHT_X, BOTTOM_RIGHT_Y = (1230, 618)
WIDTH = BOTTOM_RIGHT_X - TOP_LEFT_X
HEIGHT = BOTTOM_RIGHT_Y - TOP_LEFT_Y


In [ ]:
# # testing the coorindtes with the screenshot
# time.sleep(5)
# screenshot = pyautogui.screenshot(region=(TOP_LEFT_X, TOP_LEFT_Y, WIDTH, HEIGHT))

# screenshot.save("screenshot_region_final.png")

# print("Screenshot saved as screenshot_region.png")


# pip install pyautogui opencv-python pandas pillow keyboard

Screenshot saved as screenshot_region.png


In [ ]:
#click on the given coordinates
# x, y = 500, 300

# print("Moving to position in 3 seconds...")
# time.sleep(3)

# pyautogui.moveTo(x, y, duration=0.5) 
# pyautogui.click() 

# print(f"Clicked at ({x}, {y})")


**Scrape the Data**

In [2]:
import pyautogui
import time
import os
import sys
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
import threading

In [ ]:


try:
    from pynput import keyboard
    USING_PYNPUT = True
    print("Using pynput for keyboard interruption")
except Exception as e:
    USING_PYNPUT = False
    print(f"Could not initialize pynput: {e}")
    print("Falling back to keyboard library for interruption")
    import keyboard as kb  


running = True
stop_loop = False

MAIN_OUTPUT_DIR = "lab_tests"
os.makedirs(MAIN_OUTPUT_DIR, exist_ok=True)

def on_press(key):
    global running, stop_loop
    try:
        if key.char == 'q': 
            print("\n Script stopped by 'q' key press")
            running = False
            stop_loop = True
            return False  
    except AttributeError:
        pass  

def enhance_contrast(image_path, output_path):
    """Apply histogram equalization to enhance image contrast"""
    try:
        gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        equalized = cv2.equalizeHist(gray)
        cv2.imwrite(output_path, equalized)
        print(f"  Enhanced contrast saved as '{output_path}'")
        return output_path
    except Exception as e:
        print(f"  Error enhancing contrast: {e}")
        return image_path

def count_very_long_rows(image_path, output_path, min_line_length=500, line_gap_threshold=10):
    """Count the number of long horizontal lines in the image"""
    try:
        # Load and preprocess image
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
        
        # Morphology to enhance horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
        detect_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        
        # Find contours
        contours, _ = cv2.findContours(detect_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter lines by length
        long_line_ys = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w >= min_line_length:
                long_line_ys.append(y)
                cv2.line(img, (x, y), (x + w, y), (0, 0, 255), 2)  # Draw red line
        
        # Remove duplicates (very close Y positions)
        long_line_ys = sorted(long_line_ys)
        filtered_lines = []
        prev_y = -100
        for y in long_line_ys:
            if abs(y - prev_y) > line_gap_threshold:
                filtered_lines.append(y)
                prev_y = y
        
        # Count rows
        num_big_rows = max(len(filtered_lines) - 1, 0)
        
        # Annotate result
        cv2.putText(img, f"Very Big Rows: {num_big_rows}", (10, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)
        
        # Save
        cv2.imwrite(output_path, img)
        print(f"  Detected {num_big_rows} rows (line length ≥ {min_line_length}).")
        
        return num_big_rows
    except Exception as e:
        print(f"  Error counting rows: {e}")
        return 0

def take_screenshot(output_path):
    """Take a screenshot of the search results area"""
    try:
        screenshot = pyautogui.screenshot(region=(TOP_LEFT_X, TOP_LEFT_Y, WIDTH, HEIGHT))
        screenshot.save(output_path)
        print(f"  Screenshot saved as '{output_path}'")
        return True
    except Exception as e:
        print(f"  Error taking screenshot: {e}")
        return False

def click_on_row(row_number):
    """Click on a specific row based on its number"""
    row_y = STARTING_ROW[1] + (row_number - 1) * ROW_HEIGHT
    pyautogui.doubleClick(STARTING_ROW[0], row_y)
    print(f"  Clicked on row {row_number} at position ({STARTING_ROW[0]}, {row_y})")
    time.sleep(0.5)

def search_and_process(search_terms):
    """Process each search term from the CSV file"""
    global running

    temp_screenshot = os.path.join(MAIN_OUTPUT_DIR, "temp_results.png")
    temp_high_contrast = os.path.join(MAIN_OUTPUT_DIR, "temp_high_contrast.png")
    temp_rows_analysis = os.path.join(MAIN_OUTPUT_DIR, "temp_rows_analysis.png")
    
    for idx, term in enumerate(search_terms):
        if not running:
            print("\n Script stopped by user")
            break
            
        try:
            print(f"\n[{idx+1}/{len(search_terms)}] Processing: '{term}'")
            
            test_folder = os.path.join(MAIN_OUTPUT_DIR, term.replace("/", "_").strip())
            os.makedirs(test_folder, exist_ok=True)
            print(f"  Created folder: {test_folder}")
            
            print(f"  Clicking search bar at position {SEARCH_BAR}")
            pyautogui.moveTo(SEARCH_BAR[0], SEARCH_BAR[1], duration=0.3)
            pyautogui.click()
            time.sleep(0.3)
            
            print("  Clearing search field")
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.press('backspace')
            time.sleep(0.3)
            
            print(f"  Typing search term: {term}")
            pyautogui.write(term, interval=0.05)
            time.sleep(0.3)
            
            print("  Pressing Enter to search")
            pyautogui.press('enter')
            time.sleep(2)  
            
            take_screenshot(temp_screenshot)
            
            enhance_contrast(temp_screenshot, temp_high_contrast)
            
            row_count = count_very_long_rows(temp_high_contrast, temp_rows_analysis)
            
            current_row = 1
            while current_row <= row_count and running:
                print(f"  Processing row {current_row} of {row_count}")
                
                click_on_row(current_row)
                time.sleep(1) 
                
                detail_screenshot = os.path.join(test_folder, f"row_{current_row}.png")
                pyautogui.screenshot(detail_screenshot)
                print(f"  Saved detail view as '{detail_screenshot}'")
                
                pyautogui.moveTo(CLOSE_BUTTON[0], CLOSE_BUTTON[1], duration=0.3)
                pyautogui.click()
                time.sleep(0.5)
                
                current_row += 1
            
            print(f"  Completed processing '{term}' with {row_count} results")
            
        except Exception as e:
            print(f"  Error processing '{term}': {e}")
    
    try:
        for temp_file in [temp_screenshot, temp_high_contrast, temp_rows_analysis]:
            if os.path.exists(temp_file):
                os.remove(temp_file)
        print("Temporary analysis files cleaned up")
    except Exception as e:
        print(f"Warning: Could not clean up some temp files: {e}")
            
    print("\nAll search operations completed!")

def load_search_terms_from_csv(csv_path):
    """Load search terms from a CSV file using the 'TESTNAME' column with pandas."""
    try:
        df = pd.read_csv(csv_path)
        if 'TESTNAME' not in df.columns:
            print(f"'TESTNAME' column not found in {csv_path}")
            return []
        terms = df['TESTNAME'].dropna().astype(str).str.strip().tolist()
        print(f"Loaded {len(terms)} search terms from {csv_path}")
        return terms
    except Exception as e:
        print(f"Failed to load search terms: {e}")
        return []

def setup_keyboard_interrupt():
    """Set up keyboard interruption based on available libraries"""
    global running, stop_loop
    
    if USING_PYNPUT:
        print("Press 'q' to stop the script at any time.")
        listener = keyboard.Listener(on_press=on_press)
        listener.start()

    else:
        kb.add_hotkey('q', lambda: setattr(sys.modules[__name__], 'running', False))
        print("Press 'q' to stop the script at any time.")

def wait_for_stop_signal():
    """Simple loop that runs until stop signal is received"""
    global stop_loop, running
    
    while not stop_loop and running:
        time.sleep(0.1)
        
    return

def main():
    """Main function to run the search automation."""
    global running, stop_loop
    
    setup_keyboard_interrupt()
    
    stop_thread = threading.Thread(target=wait_for_stop_signal)
    stop_thread.daemon = True
    stop_thread.start()
    
    csv_path = "labTestName.csv"  
    search_terms = load_search_terms_from_csv(csv_path)

    if not search_terms:
        print("No search terms loaded. Make sure the CSV file and 'TESTNAME' column exist.")
        running = False
        stop_loop = True
        return
    
    print("Starting in 5 seconds. Switch to Discord now!")
    for i in range(5, 0, -1):
        print(f"{i}...")
        time.sleep(1)
        if not running:
            print("\n Script stopped before starting")
            return
    
    search_and_process(search_terms)

if __name__ == "__main__":
    try:
        try:
            import cv2
            print("OpenCV is installed - using image analysis features")
        except ImportError:
            print(" OpenCV not found. Install it with: pip install opencv-python")
            sys.exit(1)
        
        main()
        
    except KeyboardInterrupt:
        print("\n Script terminated by user")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        running = False
        stop_loop = True
        if USING_PYNPUT:
            try:
                keyboard.Listener.stop
            except:
                pass
        elif not USING_PYNPUT:
            kb.unhook_all()  

Using pynput for keyboard interruption
OpenCV is installed - using image analysis features
Press 'q' to stop the script at any time.
Loaded 1308 search terms from labTestName.csv
Starting in 5 seconds. Switch to Discord now!
5...
4...
3...

 Script stopped by 'q' key press

 Script stopped before starting
